# A Toy Trading System [Test]

I wanted to work on the node class, but that's hard without having an application/demo that is meaty enough to
work with.

So, here's a toy front-office position management, pricing, and risk system:

In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown
from mand.core import num, find
from mand.lib.dbsetup import setUpDb
import datetime

db = ObjectDb()
setUpDb(db)

## Books and Portfolios [BA]

These are just workflow entities that support a net present value function. As long as items in books are valuable
(in the sense of implementing NPV) then everything just works.

In [2]:
class TradingBook(Workbook):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            ret += i.NPV() * q
        return ret

class TradingPortfolio(Portfolio):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            ret += i.NPV() * q
        return ret
    
_tr.add(TradingBook)
_tr.add(TradingPortfolio)

## Building a tree of bank-side accounts [User]

Here, we build a tree of 100 books, grouped under 10 sub-portfolios, rooted in one top-level portfolio.

In a real bank, we might have hundreds of thousands of books, grouped 10 levels deep. in multiple different trees. 
Plus millions of customer books grouped in various portfolio trees.

In [3]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['FX', 'Rates', 'Credit', 'Eq-Prop', 'Delta1', 'Eq-Inst', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print(pAll)
print('# books:', len(pAll.books()))
print('# children:', len(pAll.children()))

<Entity:/Global/TradingPortfolio/TopOfTheHouse>
# books: 100
# children: 10


## Checking execution counts [Test]

We have 11 portfolios, 100 books. We should get node calculation counts that reflect this...

In [4]:
with ProfileMonitor(mode='sum'):
    db2 = db.copy()
    p = db2._get(pAll.meta.path())
    print(len(p.books()))

100



### Profile by nodes.
* times are in microseconds
* cumT is total time spent in function
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:books|11|483,949|118|43,995|GetValue
|Portfolio:books|11|483,938|639|43,994|GetValue/Calc
|Portfolio:children|11|483,192|118|43,926|GetValue
|Portfolio:children|11|483,073|43,599|43,915|GetValue/Calc
|PortfolioUpdateEvent:children|11|382,781|952|34,798|GetValue
|TradingBook|100|344,715|344,715|3,447|Db.Get
|PortfolioUpdateEvent|11|43,489|43,489|3,953|Db.Get
|TradingPortfolio|10|37,114|37,114|3,711|Db.Get
|Clock:cutoffs|22|8,333|72|378|GetValue
|Clock:cutoffs|1|8,261|36|8,261|GetValue/Calc
|Clock:parent|1|8,220|8|8,220|GetValue
|Clock:parent|1|8,212|4,415|8,212|GetValue/Calc
|CosmicAll|1|5,649|5,649|5,649|Db.Get
|Portfolio:clock|22|4,809|182|218|GetValue
|Portfolio:clock|11|4,626|191|420|GetValue/Calc
|Clock|1|4,435|4,435|4,435|Db.Get
|Entity:clock|2|3,703|19|1,851|GetValue
|Entity:clock|1|3,683|23|3,683|GetValue/Calc
|RootClock|1|3,660|3,660|3,660|Db.Get
|RootClock:cutoffs|3|97|15|32|GetValue
|RootClock:cutoffs|1|81|48|81|GetValue/Calc
|Event:amends|11|59|59|5|GetValue
|RootClock:cosmicAll|1|17|9|17|GetValue
|CosmicAll:dbState|1|14|8|14|GetValue
|RootClock:cosmicAll|1|8|8|8|GetValue/Calc
|CosmicAll:dbState|1|6|6|6|GetValue/Calc

## A few books [Test]

We also create two customers' books.

Pretend *p1* is one of our trading desks, *p2* is another. 

In [5]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[3]
p2 = pAll.children()[5]

b1 = p1.children()[4]
b2 = p2.children()[7]

print(bExt.meta.name())
print(b1.meta.name())
print(b2.meta.name())

Customer1
Eq-Prop4
Eq-Inst7


## Tracking Market Data [DBA]

For now, just save incoming market observations as reference data. 

In reality, we'd have billions of observations per day on millions of data sources.

In [6]:
class MarketDataSource(ExternalRefData):
    @dataField
    def last(self):
        return None
    
_tr.add(MarketDataSource)

## Making some data sources [Test]

Two pretend data sources: IBM and Google last trade prices.

In [7]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

print(s1_ibm.last())
print(s1_goog.last())

175.61
852.12


## Market Interface [DBA]

How user code accesses market data. 

In reality, market interfaces would be choosing amongst raw data sources,
providing bootstrapped curves, vol surfaces, etc.

In [8]:
class MarketInterface(Entity):
    
    @node
    def sourceName(self):
        return 'source1'
    
    @node
    def source(self):
        return self.getObj(_tr.MarketDataSource, '%s.%s' % (self.sourceName(), self.meta.name()))
    
    @node(tweakable=True)
    def spot(self):
        return self.source().last()
                           
    
_tr.add(MarketInterface)


## Making some market interfaces [Test]

In [9]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

print(ibm.spot(), type(ibm.spot()))
print(goog.spot(), type(goog.spot()))

175.61 <class 'decimal.Decimal'>
852.12 <class 'decimal.Decimal'>


## Instruments [BA]

Things we can own/have contracted. Instruments have an NPV.

We implement two classes of instrument:
* ForwardCashflow: money that will arrive at some future time
* Equity: a share of common stock in a company
    
Both implementations are toys.

In [10]:
class Instrument(WorkItem):
    """A thing that can be owned, an asset, or legal obligation"""
    
class ForwardCashflow(Instrument):
    
    @node(stored=True)
    def currency(self):
        return 'USD'
    
    @node(stored=True)
    def settlementDatetime(self):
        d = datetime.datetime.utcnow() + datetime.timedelta(2)
        return d
    
    @node
    def NPV(self):
        # XXX - would really get the currency discount curve here, and discount according to 
        # current time/settlement time
        # XXX - and do a conversion to our native currency
        return 1
    
    @node
    def name(self):
        return 'Cash %s/%s' % (self.currency(), self.settlementDatetime())
    
class Equity(Instrument):
    
    @node(stored=True)
    def assetName(self):
        return 'IBM.Eq.1'
    
    @node
    def NPV(self):
        return self.refdata().spot()
    
    @node
    def refdata(self):
        return self.getObj(_tr.MarketInterface, self.assetName().split('.')[0])

    @node
    def name(self):
        return 'Stock: %s' % self.assetName()
    
_tr.add(ForwardCashflow)
_tr.add(Equity)

## TradeOpenEvent [BA]

An event/observation that represents buying or selling something.

Someone gets *quantity* of *item*, and pays *unitPrice* of *premium* for each *item*.

Note that *premium* is typically a *ForwardCashflow*, so the money settles T+2. From a risk/PnL point
of view, we get the asset immediately. This is a gross over-simplification.

In [11]:
class TradeOpenEvent(WorkItemOpenEvent):
    @node(stored=True)
    def action(self):
        return 'Buy'
    @node(stored=True)
    def quantity(self):
        return 1.
    @node(stored=True)
    def premium(self):
        return None
    @node(stored=True)
    def unitPrice(self):
        return 0.
    
    def _items(self):
        bs = 1 if self.action() == 'Buy' else -1
        pq = -bs * self.unitPrice() * self.quantity()
        return [ [ self.ticket(), self.item(),    bs*self.quantity(), self.book1(), self.book2() ],
                 [ self.ticket(), self.premium(), pq,                 self.book1(), self.book2() ]
               ] 

_tr.add(TradeOpenEvent)

## Let's book some trades [Test]

In [12]:
with db:
    cf1 = ForwardCashflow()
    ins1 = Equity()
    ins2 = Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

# Reporting [User]

Reporting is the whole point of the Abstract Nonsense Db.

We ignore all the standard report infrastructure goop (aggregation, GUIs, pivot-tables, drill-down, etc) and
just focus on extracting the underlying data.

For the following examples, we mostly care about NPV (the present values of what we own,) 
PnL (how much money we made or lost,) and risk (what happens to our NPV if underlying conditions change.)

Note that even in the trivial case of our three trades, one amendment, and several market data updates, trying to write
code to attribute profit, or even trying to figure it by hand, would require assumptions and be error-prone. 

In general, the less a report knows about the underlying data, the better it is.

## Report: PnL [User]

How much money did we (i.e. our portfolio) gain or lose between two times?

For example, if time1 was yesterday's close, and time2 is today's close, this report tells us how much we made or lost
today. Will we be drinking champagne or straight vodka this evening?

Note: it's worth understanding how the cash entries are working in this example. The reason our open and closed out
positions are pricing rationally is because we are modelling cash approximately correctly, not because the instruments
are carrying around a notion of price-paid!

In [13]:
def reportPnL(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    with Context({clock.cutoffs: ts2}):
        npv2 = valuable.NPV()
    displayMarkdown('#### PnL for %s: %s' % (valuable.meta.name(), npv2-npv1))

### Example: top-of-the-house intraday PnL

Hmm, one of our desks bought 100 shares of IBM, and the last trade price is already down 1 cent. The bank is down 
100 times $.01 or one dollar:

In [14]:
# 100 shares bought at $175.65
# current price is $175.64:
reportPnL(pAll, ts1, ts3)

#### PnL for TopOfTheHouse: -1.00

### Example: top-of-the-house daily PnL

By end of day, the bank as a whole has closed out the IBM trade for a profit, and bought some GOOG. Closing refdata
prices have been entered. We should have a total profit for the day of $23:

In [15]:
# 100 * ($175.85 - $175.65) =  $20
# 300 * ($852.13 - $852.12) =  $ 3
#                              ----
#                              $23
reportPnL(pAll, ts1, eod)

#### PnL for TopOfTheHouse: 23.00

### Example: trading desk daily PnL

Desk *p1* bought IBM at $175.65

It still owns IBM, which closed at $175.68

So, it should be up $3:

In [16]:
# 100 * ($175.68 - $175.65) =   $3
reportPnL(p1, ts1, eod)

#### PnL for Eq-Prop: 3.00

## Report: Market Risk [DBA]

So, how much does our value change if market data values change?

Note: review the greeks workbook if this is unclear.

In [17]:
def reportMarketRisk(valuable):
    def fn(node):
        obj = node.key.object()
        m = node.key.shortName()
        if isinstance(obj, MarketInterface) and m == 'spot':
            return True
    nodes = find(valuable.NPV, fn)
    r = {}
    npv = valuable.NPV()
    for n in nodes:
        v = n.value * num(1.01)
        with Context({n: v}):
            obj = n.key.object()
            npv2 = valuable.NPV()
            r[obj.meta.name()] = npv2-npv
    displayMarkdown('### %s: NPV change under a 1%% move' % valuable.meta.name())
    displayDict(r)

In [18]:
reportMarketRisk(pAll)
reportMarketRisk(p1)
reportMarketRisk(p2)

### TopOfTheHouse: NPV change under a 1% move

|key|value|
|-|-|
|GOOG|2568.3900

### Eq-Prop: NPV change under a 1% move

|key|value|
|-|-|
|IBM|177.6800

### Eq-Inst: NPV change under a 1% move

|key|value|
|-|-|
|GOOG|2568.3900
|IBM|-177.6800

## Report: Prior Day Amends [User]

So, someone reported a number on day T. How would that number change if we reran the report based on corrected data?

There are a lot of ways to handle prior day amends:
* Have them entered, but have no systemic way to track them.
 * This is a time-honoured approach. Usually, someone goes to jail when the auditors eventually notice that some trader
declaring profits each day is just amending the trades the next day and actually losing money.
* Make a policy that there are no prior day amends.
 * Well, now at least you can see the PnL.
 * On the downside, you don't get much explanatory power. It's a bit like switching to cash accounting because your
internal business units are gaming accrual accounting to the point you can't even manage the budget.
 * Oh, and you still get fined/go to jail: if the number is your position size, and you exceeded the legal limit on
day T, you can't just pretend you didn't and book your day T over-limit trade on day T+1. External reality does not
change just because you have a bad operational policy.
* Just do it right.
 * This is the approach we take.

In [19]:
def reportRestates(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    print
    ts1r = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
    with Context({clock.cutoffs: ts1r}):
        npv2 = valuable.NPV()
    if npv2-npv1:
        displayMarkdown('#### AUDIT-ITEM. %s: NPV change due to prior day amends is %s' % 
                        (valuable.meta.name(), npv2-npv1))

In [20]:
reportRestates(p1, eod, ts6)

print()
print('Note that we are only looking at amended numbers, not all activity:')
reportPnL(p1, eod, ts6)

#### AUDIT-ITEM. Eq-Prop: NPV change due to prior day amends is -4.00


Note that we are only looking at amended numbers, not all activity:


#### PnL for Eq-Prop: 196.00

# Let's see what is going on when we run a report [Core]

In [21]:
with ProfileMonitor(mode='sum'):
    db3 = db.copy()
    p = db3._get(pAll.meta.path())
    reportPnL(p, ts1, ts6)

#### PnL for TopOfTheHouse: 1219.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in function
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Root:4561119144|516|1,263,848|783,220|2,449|Context
|TradingPortfolio:NPV|2|1,263,805|19|631,902|GetValue
|TradingPortfolio:NPV|2|1,263,785|10|631,892|GetValue/Calc
|Portfolio:items|22|1,233,065|195|56,048|GetValue
|Portfolio:items|22|1,233,049|136|56,047|GetValue/Calc
|Workbook:items|200|733,304|1,875|3,666|GetValue
|Workbook:items|200|731,428|1,276|3,657|GetValue/Calc
|Portfolio:children|44|493,799|272|11,222|GetValue
|Portfolio:children|22|493,527|142|22,433|GetValue/Calc
|PortfolioUpdateEvent:children|22|360,922|1,029|16,405|GetValue
|TradingBook|102|332,490|332,490|3,259|Db.Get
|PortfolioUpdateEvent|11|39,889|39,889|3,626|Db.Get
|TradingPortfolio|10|33,376|33,376|3,337|Db.Get
|Equity:NPV|1|30,606|10|30,606|GetValue
|Equity:NPV|1|30,596|5|30,596|GetValue/Calc
|MarketInterface:spot|1|27,521|10|27,521|GetValue
|MarketInterface:spot|1|27,510|5|27,510|GetValue/Calc
|ExternalRefData:state|1|24,534|7|24,534|GetValue
|ExternalRefData:state|1|24,527|6|24,527|GetValue/Calc
|RefData:state|1|24,495|9|24,495|GetValue
|RefData:state|1|24,486|5|24,486|GetValue/Calc
|Clock:cutoffs|446|18,244|1,416|40|GetValue
|Clock:cutoffs|5|16,827|30|3,365|GetValue/Calc
|Clock:parent|5|16,623|38|3,324|GetValue
|Clock:parent|5|16,585|29|3,317|GetValue/Calc
|TradeOpenEvent|4|14,161|14,161|3,540|Db.Get
|RefDataUpdateEvent|4|13,677|13,677|3,419|Db.Get
|Workbook:clock|400|11,323|3,325|28|GetValue
|Clock|3|10,402|10,402|3,467|Db.Get
|_WorkItemEvent:ticket|6|8,733|45|1,455|GetValue
|WorkTicket|3|8,688|8,688|2,896|Db.Get
|Workbook:clock|200|7,998|1,693|39|GetValue/Calc
|_WorkItemEvent:item|3|6,057|36|2,019|GetValue
|Equity|2|6,021|6,021|3,010|Db.Get
|_WorkItemEvent:book2|6|6,014|39|1,002|GetValue
|Portfolio:clock|44|5,256|416|119|GetValue
|Portfolio:clock|22|4,839|166|219|GetValue/Calc
|RootClock|1|4,473|4,473|4,473|Db.Get
|CosmicAll|1|3,921|3,921|3,921|Db.Get
|Equity:refdata|1|3,030|10|3,030|GetValue
|Equity:refdata|1|3,020|8|3,020|GetValue/Calc
|MarketInterface|1|2,974|2,974|2,974|Db.Get
|RefData:clock|2|2,933|14|1,466|GetValue
|TradeOpenEvent:premium|3|2,932|26|977|GetValue
|MarketInterface:source|1|2,926|10|2,926|GetValue
|RefData:clock|1|2,918|9|2,918|GetValue/Calc
|MarketInterface:source|1|2,915|5|2,915|GetValue/Calc
|ForwardCashflow|1|2,905|2,905|2,905|Db.Get
|MarketDataSource|1|2,858|2,858|2,858|Db.Get
|Portfolio:books|42|1,268|267|30|GetValue
|Portfolio:books|22|1,053|134|47|GetValue/Calc
|Entity:clock|10|181|76|18|GetValue
|Event:amends|29|151|151|5|GetValue
|Entity:clock|5|104|39|20|GetValue/Calc
|RootClock:cutoffs|15|42|42|2|GetValue
|_WorkItemEvent:book1|6|31|31|5|GetValue
|TradeOpenEvent:quantity|6|22|22|3|GetValue
|ForwardCashflow:NPV|1|22|12|22|GetValue
|MarketInterface:sourceName|1|17|8|17|GetValue
|RefDataUpdateEvent:data|4|12|12|3|GetValue
|TradeOpenEvent:action|3|12|12|4|GetValue
|TradeOpenEvent:unitPrice|3|10|10|3|GetValue
|ForwardCashflow:NPV|1|10|5|10|GetValue/Calc
|MarketInterface:sourceName|1|9|5|9|GetValue/Calc
|Equity:assetName|1|4|4|4|GetValue